In [1]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import time
import datetime
import geopandas as gpd
from datetime import date
from plotly.offline import init_notebook_mode, iplot
import plotly.express as px
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import seaborn as sns
from scipy.stats import norm
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import warnings
import os

warnings.filterwarnings('ignore')
%matplotlib inline
pd.set_option('display.max_columns', None)

In [2]:
#Imported path
path = r'/Users/james/Desktop/Task6'

In [3]:
path

'/Users/james/Desktop/Task6'

In [4]:
df = pd.read_csv('/Users/james/Desktop/Task6/immo_data.csv')

In [5]:
df.head

<bound method NDFrame.head of                      regio1  serviceCharge                     heatingType  \
0       Nordrhein_Westfalen         245.00                 central_heating   
1           Rheinland_Pfalz         134.00  self_contained_central_heating   
2                   Sachsen         255.00                   floor_heating   
3                   Sachsen          58.15                district_heating   
4                    Bremen         138.00  self_contained_central_heating   
...                     ...            ...                             ...   
268845               Bayern          90.00                       heat_pump   
268846               Hessen         220.00                     gas_heating   
268847               Hessen         220.00                 central_heating   
268848  Nordrhein_Westfalen         175.00                       heat_pump   
268849               Hessen         315.00                 central_heating   

       telekomTvOffer  telekomHyb

In [6]:
df.columns.sort_values()

Index(['balcony', 'baseRent', 'baseRentRange', 'cellar', 'condition', 'date',
       'description', 'electricityBasePrice', 'electricityKwhPrice',
       'energyEfficiencyClass', 'facilities', 'firingTypes', 'floor', 'garden',
       'geo_bln', 'geo_krs', 'geo_plz', 'hasKitchen', 'heatingCosts',
       'heatingType', 'houseNumber', 'interiorQual', 'lastRefurbish', 'lift',
       'livingSpace', 'livingSpaceRange', 'newlyConst', 'noParkSpaces',
       'noRooms', 'noRoomsRange', 'numberOfFloors', 'petsAllowed',
       'picturecount', 'pricetrend', 'regio1', 'regio2', 'regio3', 'scoutId',
       'serviceCharge', 'street', 'streetPlain', 'telekomHybridUploadSpeed',
       'telekomTvOffer', 'telekomUploadSpeed', 'thermalChar', 'totalRent',
       'typeOfFlat', 'yearConstructed', 'yearConstructedRange'],
      dtype='object')

In [7]:
# what are the different types of `regio`?
df[['regio1', 'regio2', 'regio3']].head()

,regio1,regio2,regio3
0,Nordrhein_Westfalen,Dortmund,Schüren
1,Rheinland_Pfalz,Rhein_Pfalz_Kreis,Böhl_Iggelheim
2,Sachsen,Dresden,Äußere_Neustadt_Antonstadt
3,Sachsen,Mittelsachsen_Kreis,Freiberg
4,Bremen,Bremen,Neu_Schwachhausen


In [8]:
# rename columns for regio
df.rename(columns={'regio1': 'state', 'regio2': 'district', 'regio3': 'town'}, inplace=True)

In [9]:
dat = df.query("district == 'München'")

print(f"There are {len(dat)} offers in Munich.")

dat.dtypes

There are 4383 offers in Munich.


state                        object
serviceCharge               float64
heatingType                  object
telekomTvOffer               object
telekomHybridUploadSpeed    float64
newlyConst                     bool
balcony                        bool
picturecount                  int64
pricetrend                  float64
telekomUploadSpeed          float64
totalRent                   float64
yearConstructed             float64
scoutId                       int64
noParkSpaces                float64
firingTypes                  object
hasKitchen                     bool
geo_bln                      object
cellar                         bool
yearConstructedRange        float64
baseRent                    float64
houseNumber                  object
livingSpace                 float64
geo_krs                      object
condition                    object
interiorQual                 object
petsAllowed                  object
street                       object
streetPlain                 

In [10]:
# check null values
dat.isnull().sum()

state                          0
serviceCharge                150
heatingType                 1108
telekomTvOffer               646
telekomHybridUploadSpeed    3962
newlyConst                     0
balcony                        0
picturecount                   0
pricetrend                    22
telekomUploadSpeed           647
totalRent                    440
yearConstructed              751
scoutId                        0
noParkSpaces                1939
firingTypes                 1243
hasKitchen                     0
geo_bln                        0
cellar                         0
yearConstructedRange         751
baseRent                       0
houseNumber                 1367
livingSpace                    0
geo_krs                        0
condition                   1206
interiorQual                1584
petsAllowed                 2013
street                         0
streetPlain                 1367
lift                           0
baseRentRange                  0
typeOfFlat

In [11]:
# fill nan values with average values for numerical features
for col in dat.columns:
    if dat[col].dtype == 'float':
        dat[col] = dat[col].fillna(dat[col].mean())

# fill nan values with mode for categorical features
for col in dat.columns:
    if dat[col].dtype == 'object':
        dat[col] = dat[col].fillna(dat[col].mode()[0])

In [12]:
df.head(10)

,state,serviceCharge,heatingType,telekomTvOffer,telekomHybridUploadSpeed,newlyConst,balcony,picturecount,pricetrend,telekomUploadSpeed,totalRent,yearConstructed,scoutId,noParkSpaces,firingTypes,hasKitchen,geo_bln,cellar,yearConstructedRange,baseRent,houseNumber,livingSpace,geo_krs,condition,interiorQual,petsAllowed,street,streetPlain,lift,baseRentRange,typeOfFlat,geo_plz,noRooms,thermalChar,floor,numberOfFloors,noRoomsRange,garden,livingSpaceRange,district,town,description,facilities,heatingCosts,energyEfficiencyClass,lastRefurbish,electricityBasePrice,electricityKwhPrice,date
0,Nordrhein_Westfalen,245.00,central_heating,ONE_YEAR_FREE,NaN,False,False,6,4.62,10.0,840.00,1965.0,96107057,1.0,oil,False,Nordrhein_Westfalen,True,2.0,595.00,244,86.00,Dortmund,well_kept,normal,NaN,Sch&uuml;ruferstra&szlig;e,Schüruferstraße,False,4,ground_floor,44269,4.0,181.4,1.0,3.0,4,True,4,Dortmund,Schüren,Die ebenerdig zu erreichende Erdgeschosswohnun...,Die Wohnung ist mit Laminat ausgelegt. Das Bad...,NaN,NaN,NaN,NaN,NaN,May19
1,Rheinland_Pfalz,134.00,self_contained_central_heating,ONE_YEAR_FREE,NaN,False,True,8,3.47,10.0,NaN,1871.0,111378734,2.0,gas,False,Rheinland_Pfalz,False,1.0,800.00,NaN,89.00,Rhein_Pfalz_Kreis,refurbished,normal,no,no_information,NaN,False,5,ground_floor,67459,3.0,NaN,NaN,NaN,3,False,4,Rhein_Pfalz_Kreis,Böhl_Iggelheim,Alles neu macht der Mai – so kann es auch für ...,NaN,NaN,NaN,2019.0,NaN,NaN,May19
2,Sachsen,255.00,floor_heating,ONE_YEAR_FREE,10.0,True,True,8,2.72,2.4,1300.00,2019.0,113147523,1.0,NaN,False,Sachsen,True,9.0,965.00,4,83.80,Dresden,first_time_use,sophisticated,NaN,Turnerweg,Turnerweg,True,6,apartment,1097,3.0,NaN,3.0,4.0,3,False,4,Dresden,Äußere_Neustadt_Antonstadt,Der Neubau entsteht im Herzen der Dresdner Neu...,"* 9 m² Balkon\n* Bad mit bodengleicher Dusche,...",NaN,NaN,NaN,NaN,NaN,Oct19
3,Sachsen,58.15,district_heating,ONE_YEAR_FREE,NaN,False,True,9,1.53,40.0,NaN,1964.0,108890903,NaN,district_heating,False,Sachsen,False,2.0,343.00,35,58.15,Mittelsachsen_Kreis,NaN,NaN,NaN,Gl&uuml;ck-Auf-Stra&szlig;e,Glück-Auf-Straße,False,2,other,9599,3.0,86.0,3.0,NaN,3,False,2,Mittelsachsen_Kreis,Freiberg,Abseits von Lärm und Abgasen in Ihre neue Wohn...,NaN,87.23,NaN,NaN,NaN,NaN,May19
4,Bremen,138.00,self_contained_central_heating,NaN,NaN,False,True,19,2.46,NaN,903.00,1950.0,114751222,NaN,gas,False,Bremen,False,1.0,765.00,10,84.97,Bremen,refurbished,NaN,NaN,Hermann-Henrich-Meier-Allee,Hermann-Henrich-Meier-Allee,False,5,apartment,28213,3.0,188.9,1.0,NaN,3,False,4,Bremen,Neu_Schwachhausen,Es handelt sich hier um ein saniertes Mehrfami...,Diese Wohnung wurde neu saniert und ist wie fo...,NaN,NaN,NaN,NaN,NaN,Feb20
5,Schleswig_Holstein,142.00,gas_heating,NONE,NaN,False,True,5,4.48,2.4,NaN,1999.0,115531145,NaN,gas,True,Schleswig_Holstein,False,5.0,315.20,1e,53.43,Schleswig_Flensburg_Kreis,well_kept,NaN,no,Hardeseiche,Hardeseiche,False,2,apartment,24891,2.0,165.0,NaN,NaN,2,False,2,Schleswig_Flensburg_Kreis,Struxdorf,NaN,"helle ebenerdige 2 Zi. Wohnung mit Terrasse, h...",NaN,NaN,NaN,NaN,NaN,Feb20
6,Sachsen,70.00,self_contained_central_heating,ONE_YEAR_FREE,10.0,False,False,9,1.01,2.4,380.00,NaN,114391930,NaN,NaN,False,Sachsen,True,NaN,310.00,14,62.00,Mittelsachsen_Kreis,fully_renovated,NaN,NaN,Am Bahnhof,Am_Bahnhof,False,2,NaN,9599,2.0,NaN,1.0,4.0,2,True,3,Mittelsachsen_Kreis,Freiberg,Am Bahnhof 14 in Freiberg\nHeizkosten und Warm...,NaN,NaN,NaN,NaN,NaN,NaN,Feb20
7,Bremen,88.00,central_heating,ONE_YEAR_FREE,10.0,False,True,5,1.89,2.4,584.25,1959.0,115270775,NaN,gas:electricity,False,Bremen,True,2.0,452.25,35,60.30,Bremen,NaN,NaN,NaN,Lesumer Heerstr.,Lesumer_Heerstr.,False,3,ground_floor,28717,3.0,63.0,NaN,NaN,3,False,2,Bremen,St._Magnus,+ Komfortabler Bodenbelag: Die Wohnung ist zus...,Rollläden; Warmwasserbereiter; Kellerraum; Gas...,44.00,B,NaN,NaN,NaN,Feb20
8,Baden_Württemberg,110.00,oil_heating,ONE_YEAR_FREE,NaN,False,False,5,3.77,40.0,690.00,1970.0,106416361,1.0,oil,True,Baden_Württemberg,True,2.0,580.00,NaN,53.00,Emmend

In [13]:
# check null values again
dat.isnull().sum()

state                       0
serviceCharge               0
heatingType                 0
telekomTvOffer              0
telekomHybridUploadSpeed    0
newlyConst                  0
balcony                     0
picturecount                0
pricetrend                  0
telekomUploadSpeed          0
totalRent                   0
yearConstructed             0
scoutId                     0
noParkSpaces                0
firingTypes                 0
hasKitchen                  0
geo_bln                     0
cellar                      0
yearConstructedRange        0
baseRent                    0
houseNumber                 0
livingSpace                 0
geo_krs                     0
condition                   0
interiorQual                0
petsAllowed                 0
street                      0
streetPlain                 0
lift                        0
baseRentRange               0
typeOfFlat                  0
geo_plz                     0
noRooms                     0
thermalCha

In [14]:
df.head(10)

,state,serviceCharge,heatingType,telekomTvOffer,telekomHybridUploadSpeed,newlyConst,balcony,picturecount,pricetrend,telekomUploadSpeed,totalRent,yearConstructed,scoutId,noParkSpaces,firingTypes,hasKitchen,geo_bln,cellar,yearConstructedRange,baseRent,houseNumber,livingSpace,geo_krs,condition,interiorQual,petsAllowed,street,streetPlain,lift,baseRentRange,typeOfFlat,geo_plz,noRooms,thermalChar,floor,numberOfFloors,noRoomsRange,garden,livingSpaceRange,district,town,description,facilities,heatingCosts,energyEfficiencyClass,lastRefurbish,electricityBasePrice,electricityKwhPrice,date
0,Nordrhein_Westfalen,245.00,central_heating,ONE_YEAR_FREE,NaN,False,False,6,4.62,10.0,840.00,1965.0,96107057,1.0,oil,False,Nordrhein_Westfalen,True,2.0,595.00,244,86.00,Dortmund,well_kept,normal,NaN,Sch&uuml;ruferstra&szlig;e,Schüruferstraße,False,4,ground_floor,44269,4.0,181.4,1.0,3.0,4,True,4,Dortmund,Schüren,Die ebenerdig zu erreichende Erdgeschosswohnun...,Die Wohnung ist mit Laminat ausgelegt. Das Bad...,NaN,NaN,NaN,NaN,NaN,May19
1,Rheinland_Pfalz,134.00,self_contained_central_heating,ONE_YEAR_FREE,NaN,False,True,8,3.47,10.0,NaN,1871.0,111378734,2.0,gas,False,Rheinland_Pfalz,False,1.0,800.00,NaN,89.00,Rhein_Pfalz_Kreis,refurbished,normal,no,no_information,NaN,False,5,ground_floor,67459,3.0,NaN,NaN,NaN,3,False,4,Rhein_Pfalz_Kreis,Böhl_Iggelheim,Alles neu macht der Mai – so kann es auch für ...,NaN,NaN,NaN,2019.0,NaN,NaN,May19
2,Sachsen,255.00,floor_heating,ONE_YEAR_FREE,10.0,True,True,8,2.72,2.4,1300.00,2019.0,113147523,1.0,NaN,False,Sachsen,True,9.0,965.00,4,83.80,Dresden,first_time_use,sophisticated,NaN,Turnerweg,Turnerweg,True,6,apartment,1097,3.0,NaN,3.0,4.0,3,False,4,Dresden,Äußere_Neustadt_Antonstadt,Der Neubau entsteht im Herzen der Dresdner Neu...,"* 9 m² Balkon\n* Bad mit bodengleicher Dusche,...",NaN,NaN,NaN,NaN,NaN,Oct19
3,Sachsen,58.15,district_heating,ONE_YEAR_FREE,NaN,False,True,9,1.53,40.0,NaN,1964.0,108890903,NaN,district_heating,False,Sachsen,False,2.0,343.00,35,58.15,Mittelsachsen_Kreis,NaN,NaN,NaN,Gl&uuml;ck-Auf-Stra&szlig;e,Glück-Auf-Straße,False,2,other,9599,3.0,86.0,3.0,NaN,3,False,2,Mittelsachsen_Kreis,Freiberg,Abseits von Lärm und Abgasen in Ihre neue Wohn...,NaN,87.23,NaN,NaN,NaN,NaN,May19
4,Bremen,138.00,self_contained_central_heating,NaN,NaN,False,True,19,2.46,NaN,903.00,1950.0,114751222,NaN,gas,False,Bremen,False,1.0,765.00,10,84.97,Bremen,refurbished,NaN,NaN,Hermann-Henrich-Meier-Allee,Hermann-Henrich-Meier-Allee,False,5,apartment,28213,3.0,188.9,1.0,NaN,3,False,4,Bremen,Neu_Schwachhausen,Es handelt sich hier um ein saniertes Mehrfami...,Diese Wohnung wurde neu saniert und ist wie fo...,NaN,NaN,NaN,NaN,NaN,Feb20
5,Schleswig_Holstein,142.00,gas_heating,NONE,NaN,False,True,5,4.48,2.4,NaN,1999.0,115531145,NaN,gas,True,Schleswig_Holstein,False,5.0,315.20,1e,53.43,Schleswig_Flensburg_Kreis,well_kept,NaN,no,Hardeseiche,Hardeseiche,False,2,apartment,24891,2.0,165.0,NaN,NaN,2,False,2,Schleswig_Flensburg_Kreis,Struxdorf,NaN,"helle ebenerdige 2 Zi. Wohnung mit Terrasse, h...",NaN,NaN,NaN,NaN,NaN,Feb20
6,Sachsen,70.00,self_contained_central_heating,ONE_YEAR_FREE,10.0,False,False,9,1.01,2.4,380.00,NaN,114391930,NaN,NaN,False,Sachsen,True,NaN,310.00,14,62.00,Mittelsachsen_Kreis,fully_renovated,NaN,NaN,Am Bahnhof,Am_Bahnhof,False,2,NaN,9599,2.0,NaN,1.0,4.0,2,True,3,Mittelsachsen_Kreis,Freiberg,Am Bahnhof 14 in Freiberg\nHeizkosten und Warm...,NaN,NaN,NaN,NaN,NaN,NaN,Feb20
7,Bremen,88.00,central_heating,ONE_YEAR_FREE,10.0,False,True,5,1.89,2.4,584.25,1959.0,115270775,NaN,gas:electricity,False,Bremen,True,2.0,452.25,35,60.30,Bremen,NaN,NaN,NaN,Lesumer Heerstr.,Lesumer_Heerstr.,False,3,ground_floor,28717,3.0,63.0,NaN,NaN,3,False,2,Bremen,St._Magnus,+ Komfortabler Bodenbelag: Die Wohnung ist zus...,Rollläden; Warmwasserbereiter; Kellerraum; Gas...,44.00,B,NaN,NaN,NaN,Feb20
8,Baden_Württemberg,110.00,oil_heating,ONE_YEAR_FREE,NaN,False,False,5,3.77,40.0,690.00,1970.0,106416361,1.0,oil,True,Baden_Württemberg,True,2.0,580.00,NaN,53.00,Emmend

In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 268850 entries, 0 to 268849
Data columns (total 49 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   state                     268850 non-null  object 
 1   serviceCharge             261941 non-null  float64
 2   heatingType               223994 non-null  object 
 3   telekomTvOffer            236231 non-null  object 
 4   telekomHybridUploadSpeed  45020 non-null   float64
 5   newlyConst                268850 non-null  bool   
 6   balcony                   268850 non-null  bool   
 7   picturecount              268850 non-null  int64  
 8   pricetrend                267018 non-null  float64
 9   telekomUploadSpeed        235492 non-null  float64
 10  totalRent                 228333 non-null  float64
 11  yearConstructed           211805 non-null  float64
 12  scoutId                   268850 non-null  int64  
 13  noParkSpaces              93052 non-null   f

In [16]:
df.describe()

,serviceCharge,telekomHybridUploadSpeed,picturecount,pricetrend,telekomUploadSpeed,totalRent,yearConstructed,scoutId,noParkSpaces,yearConstructedRange,baseRent,livingSpace,baseRentRange,geo_plz,noRooms,thermalChar,floor,numberOfFloors,noRoomsRange,livingSpaceRange,heatingCosts,lastRefurbish,electricityBasePrice,electricityKwhPrice
count,261941.000000,45020.0,268850.000000,267018.000000,235492.000000,2.283330e+05,211805.000000,2.688500e+05,93052.000000,211805.000000,2.688500e+05,268850.000000,268850.000000,268850.000000,268850.000000,162344.000000,217541.000000,171118.000000,268850.000000,268850.000000,85518.000000,80711.000000,46846.000000,46846.000000
mean,151.206113,10.0,9.791958,3.389001,28.804928,9.013315e+02,1966.400590,1.069697e+08,1.327634,3.714544,6.941294e+02,74.355548,3.765256,37283.022235,2.641261,114.749533,2.122405,3.572319,2.571542,3.070790,76.990866,2013.904536,89.113612,0.199769
std,308.295790,0.0,6.408399,1.964874,16.337151,3.323833e+04,46.992207,1.250093e+07,8.361403,2.738134,1.953602e+04,254.759208,2.214357,27798.037296,2.633440,61.653663,3.634934,6.375496,0.937594,1.407127,147.716278,10.963125,5.395805,0.009667
min,0.000000,10.0,0.000000,-12.330000,1.000000,0.000000e+00,1000.000000,2.887174e+07,0.000000,1.000000,0.000000e+00,0.000000,1.000000,852.000000,1.000000,0.100000,-1.000000,0.000000,1.000000,1.000000,0.000000,1015.000000,71.430000,0.170500
25%,95.000000,10.0,6.000000,2.000000,10.000000,4.698000e+02,1950.000000,1.066910e+08,1.000000,1.000000,3.380000e+02,54.000000,2.000000,9128.000000,2.000000,79.000000,1.000000,2.000000,2.000000,2.000000,54.000000,2012.000000,90.760000,0.191500
50%,135.000000,10.0,9.000000,3.390000,40.000000,6.500000e+02,1973.000000,1.111584e+08,1.000000,3.000000,4.900000e+02,67.320000,3.000000,38667.000000,3.000000,107.000000,2.000000,3.000000,3.000000,3.000000,70.000000,2017.000000,90.760000,0.198500
75%,190.000000,10.0,13.000000,4.570000,40.000000,9.850000e+02,1996.000000,1.137688e+08,1.000000,5.000000,7.990000e+02,87.000000,5.000000,57072.000000,3.000000,140.300000,3.000000,4.000000,3.000000,4.000000,90.000000,2019.000000,90.760000,0.205500
max,146118.000000,10.0,121.000000,14.920000,100.000000,1.575154e+07,2090.000000,1.157117e+08,2241.000000,9.000000,9.999999e+06,111111.000000,9.000000,99998.000000,999.990000,1996.000000,999.000000,999.000000,5.000000,7.000000,12613.000000,2919.000000,90.760000,0.227600


In [17]:
print(f'Number of columns is:',df.shape[1],'and rows is:',df.shape[0])

Number of columns is: 49 and rows is: 268850


In [18]:
df.isnull().sum()

state                            0
serviceCharge                 6909
heatingType                  44856
telekomTvOffer               32619
telekomHybridUploadSpeed    223830
newlyConst                       0
balcony                          0
picturecount                     0
pricetrend                    1832
telekomUploadSpeed           33358
totalRent                    40517
yearConstructed              57045
scoutId                          0
noParkSpaces                175798
firingTypes                  56964
hasKitchen                       0
geo_bln                          0
cellar                           0
yearConstructedRange         57045
baseRent                         0
houseNumber                  71018
livingSpace                      0
geo_krs                          0
condition                    68489
interiorQual                112665
petsAllowed                 114573
street                           0
streetPlain                  71013
lift                

In [19]:
df.drop(columns=['livingSpaceRange','street','description','facilities','geo_krs','geo_plz','scoutId','telekomUploadSpeed','telekomTvOffer','pricetrend','noRoomsRange','picturecount','geo_bln',\
    'houseNumber','streetPlain','firingTypes','yearConstructedRange'],inplace=True)

In [20]:
df.drop(columns=['garden'],inplace=True)

In [21]:
df.drop(columns=['telekomHybridUploadSpeed'],inplace=True)

In [22]:
df.drop(columns=['heatingType','noParkSpaces'],inplace=True)

In [23]:
df.drop(columns=['electricityBasePrice','electricityKwhPrice','thermalChar'],inplace=True)

In [24]:
df.drop(columns=['baseRentRange'],inplace=True)

In [25]:
df.dtypes

state                     object
serviceCharge            float64
newlyConst                  bool
balcony                     bool
totalRent                float64
yearConstructed          float64
hasKitchen                  bool
cellar                      bool
baseRent                 float64
livingSpace              float64
condition                 object
interiorQual              object
petsAllowed               object
lift                        bool
typeOfFlat                object
noRooms                  float64
floor                    float64
numberOfFloors           float64
district                  object
town                      object
heatingCosts             float64
energyEfficiencyClass     object
lastRefurbish            float64
date                      object
dtype: object

In [26]:
df.head(10)

,state,serviceCharge,newlyConst,balcony,totalRent,yearConstructed,hasKitchen,cellar,baseRent,livingSpace,condition,interiorQual,petsAllowed,lift,typeOfFlat,noRooms,floor,numberOfFloors,district,town,heatingCosts,energyEfficiencyClass,lastRefurbish,date
0,Nordrhein_Westfalen,245.00,False,False,840.00,1965.0,False,True,595.00,86.00,well_kept,normal,NaN,False,ground_floor,4.0,1.0,3.0,Dortmund,Schüren,NaN,NaN,NaN,May19
1,Rheinland_Pfalz,134.00,False,True,NaN,1871.0,False,False,800.00,89.00,refurbished,normal,no,False,ground_floor,3.0,NaN,NaN,Rhein_Pfalz_Kreis,Böhl_Iggelheim,NaN,NaN,2019.0,May19
2,Sachsen,255.00,True,True,1300.00,2019.0,False,True,965.00,83.80,first_time_use,sophisticated,NaN,True,apartment,3.0,3.0,4.0,Dresden,Äußere_Neustadt_Antonstadt,NaN,NaN,NaN,Oct19
3,Sachsen,58.15,False,True,NaN,1964.0,False,False,343.00,58.15,NaN,NaN,NaN,False,other,3.0,3.0,NaN,Mittelsachsen_Kreis,Freiberg,87.23,NaN,NaN,May19
4,Bremen,138.00,False,True,903.00,1950.0,False,False,765.00,84.97,refurbished,NaN,NaN,False,apartment,3.0,1.0,NaN,Bremen,Neu_Schwachhausen,NaN,NaN,NaN,Feb20
5,Schleswig_Holstein,142.00,False,True,NaN,1999.0,True,False,315.20,53.43,well_kept,NaN,no,False,apartment,2.0,NaN,NaN,Schleswig_Flensburg_Kreis,Struxdorf,NaN,NaN,NaN,Feb20
6,Sachsen,70.00,False,False,380.00,NaN,False,True,310.00,62.00,fully_renovated,NaN,NaN,False,NaN,2.0,1.0,4.0,Mittelsachsen_Kreis,Freiberg,NaN,NaN,NaN,Feb20
7,Bremen,88.00,False,True,584.25,1959.0,False,True,452.25,60.30,NaN,NaN,NaN,False,ground_floor,3.0,NaN,NaN,Bremen,St._Magnus,44.00,B,NaN,Feb20
8,Baden_Württemberg,110.00,False,False,690.00,1970.0,True,True,580.00,53.00,well_kept,sophisticated,no,False,roof_storey,2.0,2.0,2.0,Emmendingen_Kreis,Denzlingen,NaN,E,NaN,Feb20
9,Nordrhein_Westfalen,95.00,False,False,NaN,1953.0,False,True,300.00,60.00,well_kept,normal,negotiable,False,apartment,2.5,2.0,5.0,Gelsenkirchen,Bulmke_Hüllen,NaN,NaN,NaN,Oct19


In [27]:
def missing_values(df,norows):   # input by the df and the number of rows that you want to show
    total = df.isnull().sum().sort_values(ascending=False)
    percent = ((df.isnull().sum().sort_values(ascending=False)/df.shape[0])*100).sort_values(ascending=False)
    missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
    return(missing_data.head(norows))

In [28]:
missing_values(df,20) # we use the df and the number of rows to show is 20

,Total,Percent
energyEfficiencyClass,191063,71.066766
lastRefurbish,188139,69.979171
heatingCosts,183332,68.191185
petsAllowed,114573,42.615957
interiorQual,112665,41.906267
numberOfFloors,97732,36.351869
condition,68489,25.474800
yearConstructed,57045,21.218151
floor,51309,19.084620
totalRent,40517,15.070485


In [29]:
missing_data = missing_values(df,20)
# drop the data where the columns contains more than 16%
df = df.drop((missing_data[missing_data['Percent'] > 16]).index,1)

In [30]:
    df.head()

,state,serviceCharge,newlyConst,balcony,totalRent,hasKitchen,cellar,baseRent,livingSpace,lift,typeOfFlat,noRooms,district,town,date
0,Nordrhein_Westfalen,245.00,False,False,840.0,False,True,595.0,86.00,False,ground_floor,4.0,Dortmund,Schüren,May19
1,Rheinland_Pfalz,134.00,False,True,NaN,False,False,800.0,89.00,False,ground_floor,3.0,Rhein_Pfalz_Kreis,Böhl_Iggelheim,May19
2,Sachsen,255.00,True,True,1300.0,False,True,965.0,83.80,True,apartment,3.0,Dresden,Äußere_Neustadt_Antonstadt,Oct19
3,Sachsen,58.15,False,True,NaN,False,False,343.0,58.15,False,other,3.0,Mittelsachsen_Kreis,Freiberg,May19
4,Bremen,138.00,False,True,903.0,False,False,765.0,84.97,False,apartment,3.0,Bremen,Neu_Schwachhausen,Feb20


In [31]:
df.isnull().sum()

state                0
serviceCharge     6909
newlyConst           0
balcony              0
totalRent        40517
hasKitchen           0
cellar               0
baseRent             0
livingSpace          0
lift                 0
typeOfFlat       36614
noRooms              0
district             0
town                 0
date                 0
dtype: int64

In [32]:
df.info

<bound method DataFrame.info of                       state  serviceCharge  newlyConst  balcony  totalRent  \
0       Nordrhein_Westfalen         245.00       False    False      840.0   
1           Rheinland_Pfalz         134.00       False     True        NaN   
2                   Sachsen         255.00        True     True     1300.0   
3                   Sachsen          58.15       False     True        NaN   
4                    Bremen         138.00       False     True      903.0   
...                     ...            ...         ...      ...        ...   
268845               Bayern          90.00       False     True      910.0   
268846               Hessen         220.00       False     True     1150.0   
268847               Hessen         220.00       False     True      930.0   
268848  Nordrhein_Westfalen         175.00        True     True     1015.0   
268849               Hessen         315.00       False     True        NaN   

        hasKitchen  cellar  bas

In [33]:
regionlist = list(df['district'].value_counts().head(25).index)
regionlist # top 20 city that contains the most data counts

['Leipzig',
 'Chemnitz',
 'Berlin',
 'Dresden',
 'Magdeburg',
 'Halle_Saale',
 'München',
 'Essen',
 'Frankfurt_am_Main',
 'Hamburg',
 'Düsseldorf',
 'Duisburg',
 'Mittelsachsen_Kreis',
 'Dortmund',
 'Gelsenkirchen',
 'Recklinghausen_Kreis',
 'Köln',
 'Zwickau',
 'Wuppertal',
 'Zwickau_Kreis',
 'Leipzig_Kreis',
 'Meißen_Kreis',
 'Sächsische_Schweiz_Osterzgebirge_Kreis',
 'Bochum',
 'Bremen']

In [34]:
df = df[pd.DataFrame(df['district'].tolist()).isin(regionlist).any(1).values]
df['district'].value_counts()

Leipzig                                   13723
Chemnitz                                  12575
Berlin                                    10406
Dresden                                    7522
Magdeburg                                  4860
Halle_Saale                                4565
München                                    4383
Essen                                      4351
Frankfurt_am_Main                          4296
Hamburg                                    3759
Düsseldorf                                 3711
Duisburg                                   3522
Mittelsachsen_Kreis                        3162
Dortmund                                   3137
Gelsenkirchen                              2943
Recklinghausen_Kreis                       2754
Köln                                       2709
Zwickau                                    2536
Wuppertal                                  2376
Zwickau_Kreis                              2321
Leipzig_Kreis                           

In [35]:
df['baseRent'].describe().round(2)

count    110200.00
mean        681.61
std         828.13
min           0.00
25%         321.15
50%         450.00
75%         810.00
max      120000.00
Name: baseRent, dtype: float64

In [36]:
df['totalRent'].describe().round(2)

count    96905.00
mean       853.24
std        751.66
min          0.00
25%        452.13
50%        610.00
75%       1000.00
max      63204.00
Name: totalRent, dtype: float64

In [37]:
df.head()

,state,serviceCharge,newlyConst,balcony,totalRent,hasKitchen,cellar,baseRent,livingSpace,lift,typeOfFlat,noRooms,district,town,date
0,Nordrhein_Westfalen,245.00,False,False,840.0,False,True,595.0,86.00,False,ground_floor,4.0,Dortmund,Schüren,May19
2,Sachsen,255.00,True,True,1300.0,False,True,965.0,83.80,True,apartment,3.0,Dresden,Äußere_Neustadt_Antonstadt,Oct19
3,Sachsen,58.15,False,True,NaN,False,False,343.0,58.15,False,other,3.0,Mittelsachsen_Kreis,Freiberg,May19
4,Bremen,138.00,False,True,903.0,False,False,765.0,84.97,False,apartment,3.0,Bremen,Neu_Schwachhausen,Feb20
6,Sachsen,70.00,False,False,380.0,False,True,310.0,62.00,False,NaN,2.0,Mittelsachsen_Kreis,Freiberg,Feb20


In [38]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 110200 entries, 0 to 268849
Data columns (total 15 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   state          110200 non-null  object 
 1   serviceCharge  108471 non-null  float64
 2   newlyConst     110200 non-null  bool   
 3   balcony        110200 non-null  bool   
 4   totalRent      96905 non-null   float64
 5   hasKitchen     110200 non-null  bool   
 6   cellar         110200 non-null  bool   
 7   baseRent       110200 non-null  float64
 8   livingSpace    110200 non-null  float64
 9   lift           110200 non-null  bool   
 10  typeOfFlat     94566 non-null   object 
 11  noRooms        110200 non-null  float64
 12  district       110200 non-null  object 
 13  town           110200 non-null  object 
 14  date           110200 non-null  object 
dtypes: bool(5), float64(5), object(5)
memory usage: 9.8+ MB


In [39]:
# rename columns 
df.rename(columns={'noRooms': 'Number of Rooms', 'newlyConst': 'New Build'}, inplace=True)

In [40]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 110200 entries, 0 to 268849
Data columns (total 15 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   state            110200 non-null  object 
 1   serviceCharge    108471 non-null  float64
 2   New Build        110200 non-null  bool   
 3   balcony          110200 non-null  bool   
 4   totalRent        96905 non-null   float64
 5   hasKitchen       110200 non-null  bool   
 6   cellar           110200 non-null  bool   
 7   baseRent         110200 non-null  float64
 8   livingSpace      110200 non-null  float64
 9   lift             110200 non-null  bool   
 10  typeOfFlat       94566 non-null   object 
 11  Number of Rooms  110200 non-null  float64
 12  district         110200 non-null  object 
 13  town             110200 non-null  object 
 14  date             110200 non-null  object 
dtypes: bool(5), float64(5), object(5)
memory usage: 9.8+ MB


In [41]:
df.rename(columns={'state': 'State', 'serviceCharge': 'Service Charge', 'livingSpace': 'Living Space'}, inplace=True)

In [42]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 110200 entries, 0 to 268849
Data columns (total 15 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   State            110200 non-null  object 
 1   Service Charge   108471 non-null  float64
 2   New Build        110200 non-null  bool   
 3   balcony          110200 non-null  bool   
 4   totalRent        96905 non-null   float64
 5   hasKitchen       110200 non-null  bool   
 6   cellar           110200 non-null  bool   
 7   baseRent         110200 non-null  float64
 8   Living Space     110200 non-null  float64
 9   lift             110200 non-null  bool   
 10  typeOfFlat       94566 non-null   object 
 11  Number of Rooms  110200 non-null  float64
 12  district         110200 non-null  object 
 13  town             110200 non-null  object 
 14  date             110200 non-null  object 
dtypes: bool(5), float64(5), object(5)
memory usage: 9.8+ MB


In [43]:
df.info

<bound method DataFrame.info of                       State  Service Charge  New Build  balcony  totalRent  \
0       Nordrhein_Westfalen          245.00      False    False      840.0   
2                   Sachsen          255.00       True     True     1300.0   
3                   Sachsen           58.15      False     True        NaN   
4                    Bremen          138.00      False     True      903.0   
6                   Sachsen           70.00      False    False      380.0   
...                     ...             ...        ...      ...        ...   
268842              Sachsen          140.00      False    False      440.0   
268843              Sachsen          120.00      False     True      368.0   
268844  Nordrhein_Westfalen           80.00      False    False      670.0   
268848  Nordrhein_Westfalen          175.00       True     True     1015.0   
268849               Hessen          315.00      False     True        NaN   

        hasKitchen  cellar  bas

In [47]:
df.to_pickle("./dummy.pkl")

In [49]:
df.to_pickle(os.path.join(path, 'Rental.pkl'))